In [20]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [6]:
executable_path = {"executable_path": "C:/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [7]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)



# NASA Mars News

In [24]:
for item in range(1,50):
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    results = soup.find("ul", class_="item_list")
    time.sleep(1)
    
    for article in results:
        news_title = article.find("div", class_="content_title").text
        news_p = article.find("div", class_="article_teaser_body").text
        
        print(news_title)
        print(news_p)
    browser.click_link_by_partial_text('More')

The MarCO Mission Comes to an End
The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.
A Year of Surprising Science From NASA's InSight Mars Mission
A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet.
Mars InSight Lander to Push on Top of the 'Mole'
Engineers have a plan for pushing down on the heat probe, which has been stuck at the Martian surface for a year.
NASA Adds Return Sample Scientists to Mars 2020 Leadership Team
The leadership council for Mars 2020 science added two new members who represent the interests of scientists destined to handle and study the first samples from Red Planet.
NASA's Mars Reconnaissance Orbiter Undergoes Memory Update
Other orbiters will continue relaying data from Mars surface missions for a two-week period.
NASA's Mars 2020 Rover Goes Coast-to-Coast to Prep for Launch
The agency's first step in returning rocks from Mars just arrived at Kennedy Space Center. The Mars 

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=80.0.3987.122)


In [11]:
results

In [ ]:
for x in range(1, 6):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    quotes = soup.find_all('span', class_='text')

    for quote in quotes:
        print('page:', x, '-------------')
        print(quote.text)

    browser.click_link_by_partial_text('Next')

In [ ]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."

# JPL Mars Space Images - Featured Image

In [ ]:
# Visit the url for JPL Featured Space Image here.
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.

# Example:
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'


# Mars Weather

In [ ]:
# Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.
# Example:
mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'


# Mars Facts

In [ ]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
# Use Pandas to convert the data to a HTML table string.


# Mars Hemispheres

In [ ]:

# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.